# 4th‑Down Decision Model — My Project

This notebook contains my end‑to‑end work on predicting fourth‑down conversion probability. I load a cleaned snapshot of play‑by‑play data, create only pre‑play features (no leakage), fit and evaluate classifiers, and save a final model and scaler for interactive use in the dashboard. The final decision threshold I use is 0.45 (45%): probability >= 0.45 → recommend "go for it".

## 1 — Setup and imports

I import the standard libraries I need for data wrangling, modeling and evaluation. I set a fixed random seed for reproducible results during development and testing.

In [35]:
# Core imports used throughout the notebook
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Set a stable random seed for reproducible results while demonstrating functionality
RANDOM_SEED = 2023

## 2 — Load and clean real NFL play‑by‑play data (nfldata.csv)

I load a targeted subset of the full play‑by‑play CSV (only the columns I expect to need) so the notebook stays responsive while I work interactively. My cleaning steps are intentional and conservative — I keep only plays that are realistic candidates for a "go for it" decision and I avoid any post‑play information that would leak the outcome into features.

What I do and why:

- Read only the columns I need (keeps memory usage low). If the CSV changes shape this step is resilient: I check which requested columns are available and proceed with the intersection.
- Filter to 4th‑down plays and keep only offensive pass/run attempts — this restricts the dataset to real decisions where "go for it" is applicable (teams otherwise might punt, kick or kneel).
- Keep only rows that have a definitive `fourth_down_converted` label (our supervised target). Ambiguous or missing labels are dropped because they can't be used for learning.
- Drop rows missing the minimal pre‑play numeric fields we rely on (`ydstogo`, `yardline_100`). These fields are essential for model inputs; rather than guessing, I conservatively remove incomplete rows.
- Convert the target to integer (0/1) to ensure consistent training behavior.
- Convert boolean‑like indicators (`shotgun`, `no_huddle`, `qb_dropback`) to numeric 0/1 to make them usable by sklearn models.
- For categorical fields like `pass_length` and `pass_location`, replace missing values with the string `'NA'` and one‑hot encode them to preserve situational signals without introducing leakage.

Notes / limitations:
- This pipeline is deliberately conservative: I prefer dropping a few ambiguous rows rather than risk leakage or incorrect labels. If you want to keep more rows, we can add imputation rules or a more nuanced missing‑value strategy.
- The selection of columns and pre‑play features is flexible — I document the set of 'base' features used and the pipeline so you (or others) can adapt it to different datasets or include additional situational fields.
- Because the CSV is large, these steps balance correctness with interactivity; running the full pipeline in a batch job (or on a more powerful machine) is an easy next step if you want to expand the feature set or tune imputation.

In [36]:
# Load a *focused subset* of columns from the CSV to keep memory usage low while working interactively.
# We only load columns we expect to need for modeling and cleaning.
csv_path = '../nfldata.csv'  # relative path from this notebook's directory

# Columns we will read and consider as candidate features or metadata
read_cols = [
	'down', 'play_type', 'fourth_down_converted', 'ydstogo', 'yardline_100', 'qtr',
	'game_seconds_remaining', 'score_differential', 'shotgun', 'no_huddle', 'qb_dropback',
	'pass_length', 'pass_location'
]

# Read the CSV header to compute which of our requested columns actually exist in this file
# (safety for different dataset versions).
header = pd.read_csv(csv_path, nrows=0)
available = [c for c in read_cols if c in header.columns]
print('Available columns found and loaded:', available)

# Read only the columns that are available. The full file is large; limiting columns helps memory.
df_raw = pd.read_csv(csv_path, usecols=available, low_memory=True)

# --- Cleaning steps and selection ---
# 1) Focus on 4th-down plays only — these are the situations where the "go for it" decision applies.
df = df_raw[df_raw['down'] == 4]

# 2) We will only model *offensive attempts* on 4th down that are realistic "go for it" candidates.
#    In practice teams either attempt a pass/run, or they choose to punt/field_goal/kneel. We restrict
#    to 'pass' and 'run' play types to model the conversion probability of an attempt to gain the first.
df = df[df['play_type'].isin(['pass', 'run'])]

# 3) Keep rows where `fourth_down_converted` (our target) is present — remove ambiguous / NA rows.
df = df[df['fourth_down_converted'].notnull()].copy()

# 4) Remove any rows missing the minimal set of pre-play numeric information (a simple, conservative rule).
df = df[df['ydstogo'].notnull() & df['yardline_100'].notnull()]

# 5) Convert target to integer (0/1) and map float-ish values to ints safely
df['fourth_down_converted'] = df['fourth_down_converted'].astype(int)

# Print a quick summary so you can confirm cleaning progress
print('\nAfter filtering to 4th-down offensive attempts (pass/run) with target available:')
print(' rows:', len(df))
print(' converted distribution:')
print(df['fourth_down_converted'].value_counts().to_dict())

# 6) Quick check for remaining missingness in columns we plan to use
print('\nMissing values by column (after filters):')
print(df[available].isnull().sum())

# Show the first few rows of the cleaned dataset so you can verify everything looks right
df.head()

Available columns found and loaded: ['down', 'play_type', 'fourth_down_converted', 'ydstogo', 'yardline_100', 'qtr', 'game_seconds_remaining', 'score_differential', 'shotgun', 'no_huddle', 'qb_dropback', 'pass_length', 'pass_location']

After filtering to 4th-down offensive attempts (pass/run) with target available:
 rows: 4721
 converted distribution:
{0: 2383, 1: 2338}

Missing values by column (after filters):
down                         0
play_type                    0
fourth_down_converted        0
ydstogo                      0
yardline_100                 0
qtr                          0
game_seconds_remaining       0
score_differential           0
shotgun                      0
no_huddle                    0
qb_dropback                  0
pass_length               1936
pass_location             1936
dtype: int64

After filtering to 4th-down offensive attempts (pass/run) with target available:
 rows: 4721
 converted distribution:
{0: 2383, 1: 2338}

Missing values by column (af

,yardline_100,game_seconds_remaining,qtr,down,ydstogo,play_type,shotgun,no_huddle,qb_dropback,pass_length,pass_location,score_differential,fourth_down_converted
329,64.0,292.0,4,4.0,10,pass,1,1,1.0,short,middle,-22.0,0
395,4.0,2651.0,2,4.0,1,run,0,0,0.0,NaN,NaN,-11.0,1
516,64.0,378.0,4,4.0,1,pass,0,0,1.0,short,right,-19.0,1
676,26.0,464.0,4,4.0,7,pass,1,0,1.0,short,middle,-13.0,0
697,2.0,91.0,4,4.0,2,pass,0,0,1.0,short,right,-20.0,1


## 3 — Train a classifier (Logistic Regression)

I start with Logistic Regression as my interpretable baseline and document the steps carefully so the model decisions are explainable during presentation.

Key choices and why:
- Model: Logistic Regression (binary) — I chose it because it provides direct, interpretable coefficients and outputs well-calibrated probabilities, which I can turn into a go/no‑go recommendation using a probability threshold.
- Train/test split: I reserve 25% of the cleaned data for testing and use stratified splits so the class balance (converted vs not converted) remains consistent across folds and held‑out evaluation.
- Scaling: I standardize all features using a StandardScaler. Because I use one‑hot encodings for categorical fields and numeric variables with different units/ranges, scaling ensures the regularization and coefficients behave sensibly.
- Regularization & solver: The implementation uses liblinear (L2 by default). Regularization prevents overfitting when features are correlated and keeps the learned coefficients stable.

How I convert probability into a decision:
- The model returns a probability that the 4th‑down attempt converts. I use a decision threshold of 0.45 — probability >= 0.45 → recommend GO; otherwise NO GO. I chose 0.45 to reflect the project’s conservative decisioning policy (closer to the calibrated trade‑off we want), but the threshold is easy to adjust during analysis or production.

Evaluation metrics I report:
- Accuracy: overall agreement with true labels (useful for broad sense of correctness, but can hide class imbalance behavior).
- Precision: of the plays predicted as conversions, how many actually converted (reduces false positives — important when avoiding risky calls).
- Recall: of the true conversions, how many were correctly identified (helps understand missed conversion opportunities).
- Confusion matrix: a compact view of true/false positives and negatives so I can explain practical impacts (e.g., how often model recommends a risky 'go' that fails).

This baseline gives me a transparent tool to explain the model’s behavior, compare more complex models against it, and inspect which features the model is relying on by examining coefficient magnitudes and signs.

In [37]:
# --- Build final feature matrix for modeling (careful: no post-play leakage) ---
# We choose pre-play features only: distance-to-get (`ydstogo`), field position (`yardline_100` — distance from opponent endzone),
# quarter (`qtr`), seconds remaining in the game (`game_seconds_remaining`), score differential and simple indicators such as
# whether the offense lined up in shotgun, no-huddle or used a QB dropback flag. We also include lightweight categorical flags
# extracted from 'pass_length' and 'pass_location' when available (run plays often have NA for these but we keep NA as a valid category).

TARGET = 'fourth_down_converted'

# Start from a conservative list of candidate columns we loaded earlier. We'll one-hot encode pass_length and pass_location.
base_features = ['ydstogo', 'yardline_100', 'qtr', 'game_seconds_remaining', 'score_differential', 'shotgun', 'no_huddle', 'qb_dropback']
cat_features = [c for c in ['pass_length', 'pass_location'] if c in df.columns]

# Keep only rows with numeric features available — we already filtered some above, but be conservative.
df_model = df.dropna(subset=['ydstogo', 'yardline_100'])

# Convert boolean-like columns to numeric (some files are already numeric, but we enforce dtype)
for col in ['shotgun', 'no_huddle', 'qb_dropback']:
	if col in df_model.columns:
		df_model[col] = pd.to_numeric(df_model[col], errors='coerce').fillna(0).astype(int)

# Convert categorical NA into the string 'NA' so it's represented as a proper category in get_dummies
for c in cat_features:
	df_model[c] = df_model[c].fillna('NA').astype(str)

# Create the full feature frame with one-hot encoded categorical columns
X = df_model[base_features].copy()
if cat_features:
	X = pd.concat([X, pd.get_dummies(df_model[cat_features], prefix=cat_features, dummy_na=False)], axis=1)

# Target vector
y = df_model[TARGET].astype(int)

# Show feature count and a quick preview for debugging
print('\nPrepared modeling dataset:')
print(' rows:', len(X), 'features:', X.shape[1])
X.head()

# If a features list was produced by the selection utility, prefer that ordering and subset X
import os, joblib
features_file = os.path.join(os.path.dirname(os.path.abspath('')), 'models', 'features.joblib')
if os.path.exists(features_file):
    try:
        _selected = joblib.load(features_file)
        present = [f for f in _selected if f in X.columns]
        if len(present) > 0:
            X = X[present].copy()
            print('Subsetting X to selected features (from models/features.joblib):', present)
    except Exception as _exc:
        print('Failed to load models/features.joblib — proceeding with original features:', _exc)

# Train / test split — stratify on y to preserve conversion proportion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED, stratify=y)

# Scale numeric features. Because we used one-hot encodings, scaling everything is acceptable for logistic regression.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression. We set class_weight=None because the filtered sample is already fairly balanced; adjust if using larger datasets.
model = LogisticRegression(random_state=RANDOM_SEED, solver='liblinear', max_iter=1000)
model.fit(X_train_scaled, y_train)

# Predictions and evaluation using 0.45 threshold
y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]
y_pred_label = (y_pred_prob >= 0.45).astype(int)

acc = accuracy_score(y_test, y_pred_label)
prec = precision_score(y_test, y_pred_label, zero_division=0)
rec = recall_score(y_test, y_pred_label, zero_division=0)
cm = confusion_matrix(y_test, y_pred_label)

print("Model evaluation (threshold = 0.45):")
print(f" Accuracy: {acc:.3f}")
print(f" Precision: {prec:.3f}")
print(f" Recall: {rec:.3f}")
print("Confusion matrix:\n", cm)


Prepared modeling dataset:
 rows: 4721 features: 15
Model evaluation (threshold = 0.45):
 Accuracy: 0.605
 Precision: 0.577
 Recall: 0.762
Confusion matrix:
 [[269 327]
 [139 446]]
Model evaluation (threshold = 0.45):
 Accuracy: 0.605
 Precision: 0.577
 Recall: 0.762
Confusion matrix:
 [[269 327]
 [139 446]]


## 3b — Model selection and final training (detailed)

I compare two practical approaches and pick the one that best generalizes to unseen plays:

What I compare
- Logistic Regression (regularized) — interpretable, gives coefficients that are easy to explain and reason about.
- Random Forest — often picks up complex, non‑linear interactions and is robust to noisy features.

Search & validation procedure
- I run a limited grid search over sensible hyperparameter values for both estimators. For Logistic Regression I tune regularization strength and class weighting; for Random Forest I tune  n_estimators, depth and class weight.
- I use 5‑fold stratified cross‑validation and ROC AUC as the selection metric. Stratified folds keep the conversion rate consistent across folds, and AUC focuses on the model’s ranking ability which is often more informative than raw accuracy for probability‐based decisioning.
- This cross‑validated selection helps me avoid choosing a model that overfits a specific split or favours an arbitrary metric.

Final training
- Once I pick the best estimator by CV AUC, I retrain it on the entire cleaned dataset so the final model benefits from all available labeled data.
- I fit a StandardScaler on the full feature matrix and transform features before the final fit — this ensures the saved scaler and model are consistent and can be reused in the dashboard and helpers.

Why these choices
- The logistic baseline gives me a clear, explainable model I can present and use to show how particular features change the odds of success (e.g., how yards to go affects conversion probability).
- The Random Forest is a complementary check for performance. If it substantially improves AUC I use it instead — but only after validating the improvement with cross‑validation and inspecting feature importances so the decision is explainable.

Saved artifacts
- I save the chosen estimator and the scaler so downstream code (dashboard and prediction helpers) can load and use them without retraining.
- I also save the final ordered feature list (features.joblib) used for training so the dashboard and single‑play helpers can construct inputs in the exact same order — that avoids feature-mismatch problems at prediction time.

Practical notes
- If we need stricter model interpretability in a production setting, the logistic model remains an excellent option (it’s transparent and easy to audit). If we need additional accuracy and the Random Forest is measurably better, I’ll keep it but document the explanation strategy (e.g., top feature importances and partial dependence where necessary).
- Everything here is configured to be repeatable: the search, selection metric and final training are deterministic given the fixed random seed so you can reproduce the same final artifacts anytime.

In [38]:
# Model optimization: compare LogisticRegression (L2) and a RandomForest and select the best model by ROC AUC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
import joblib
import os

# We'll use stratified folds to preserve labels across splits
search_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

# Logistic grid (allow class_weight option to handle any imbalance)
log_grid = {
    'C': [0.01, 0.1, 1, 10],
    'class_weight': [None, 'balanced']
}
log_clf = LogisticRegression(solver='liblinear', random_state=RANDOM_SEED, max_iter=2000)
log_search = GridSearchCV(log_clf, log_grid, scoring='roc_auc', cv=search_cv, n_jobs=-1, verbose=1)
log_search.fit(X_train_scaled, y_train)
print('Logistic best:', log_search.best_params_, 'AUC:', log_search.best_score_)

# RandomForest grid
rf_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'class_weight': [None, 'balanced']
}
rf = RandomForestClassifier(random_state=RANDOM_SEED)
rf_search = GridSearchCV(rf, rf_grid, scoring='roc_auc', cv=search_cv, n_jobs=-1, verbose=1)
rf_search.fit(X_train_scaled, y_train)
print('RandomForest best:', rf_search.best_params_, 'AUC:', rf_search.best_score_)

# Choose the better model by cross-validated AUC
if rf_search.best_score_ > log_search.best_score_:
    best_estimator = rf_search.best_estimator_
    best_name = 'RandomForest'
    best_score = rf_search.best_score_
else:
    best_estimator = log_search.best_estimator_
    best_name = 'LogisticRegression'
    best_score = log_search.best_score_

print(f'Best model chosen: {best_name} with cross-validated ROC AUC = {best_score:.4f}')

# Evaluate the chosen model on the entire dataset using cross-validation for transparency
cv_auc = cross_val_score(best_estimator, X, y, cv=search_cv, scoring='roc_auc', n_jobs=-1)
print('Cross-validated AUC (folds):', cv_auc, 'mean:', cv_auc.mean())

# Train the chosen model on the entire cleaned dataset (final model)
scaler_full = StandardScaler().fit(X)
X_full_scaled = scaler_full.transform(X)
best_estimator.fit(X_full_scaled, y)

# Save model and scaler for later use by dashboard or scripts
models_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'models')
os.makedirs(models_dir, exist_ok=True)
joblib.dump(best_estimator, os.path.join(models_dir, 'final_model.joblib'))
joblib.dump(scaler_full, os.path.join(models_dir, 'scaler.joblib'))
# Save the feature ordering used to train the model so other scripts/apps can load it (features.joblib)
joblib.dump(X.columns.tolist(), os.path.join(models_dir, 'features.joblib'))
print('Saved final_model.joblib and scaler.joblib to', models_dir)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Logistic best: {'C': 0.1, 'class_weight': None} AUC: 0.6941743907354597
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Logistic best: {'C': 0.1, 'class_weight': None} AUC: 0.6941743907354597
Fitting 5 folds for each of 12 candidates, totalling 60 fits
RandomForest best: {'class_weight': None, 'max_depth': 10, 'n_estimators': 100} AUC: 0.7027121472791317
Best model chosen: RandomForest with cross-validated ROC AUC = 0.7027
RandomForest best: {'class_weight': None, 'max_depth': 10, 'n_estimators': 100} AUC: 0.7027121472791317
Best model chosen: RandomForest with cross-validated ROC AUC = 0.7027
Cross-validated AUC (folds): [0.68735553 0.7283993  0.6923424  0.67098506 0.66975957] mean: 0.6897683727333807
Cross-validated AUC (folds): [0.68735553 0.7283993  0.6923424  0.67098506 0.66975957] mean: 0.6897683727333807
Saved final_model.joblib and scaler.joblib to c:\Users\owens\OneDrive\Desktop\BHDAC\models
Saved final_m

## 3c — Model diagnostics and interactive visuals

I run a few concise diagnostics and interactive charts to validate model behavior and prepare the visuals I'll use in the demo.

What I produce and why:
- Cross‑validated ROC curve (AUC): shows the model's ranking ability on unseen data. I use cross‑validated probabilities so the curve reflects generalization, not a single split.
- Predicted probability distribution (by true label): lets me inspect separation between conversions and failures, check for overlapping mass, and spot over‑ or under‑confidence.
- Top features (coefficients or importances): for logistic regression I show signed coefficients (direction + magnitude); for RandomForest I show feature_importances_. This helps explain which features drive the model's decisions.

Quick checks I perform:
- Confirm the model isn't overconfident (look for probability mass excessively near 0 or 1).
- Inspect misclassified / borderline examples to understand common failure modes.
- Verify important features align with domain intuition (e.g., ydstogo and yardline_100 behavior).

Interactive notes:
- Charts use Plotly so I can hover, zoom, and highlight during the presentation to demonstrate specific cases and explain model behavior.

In [39]:
# Diagnostics & Plotly charts (requires plotly installed)
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc, confusion_matrix

# Cross-validated probabilities via cross_val_predict to get more robust probability estimates
from sklearn.model_selection import cross_val_predict
probs_cv = cross_val_predict(best_estimator, X, y, cv=search_cv, method='predict_proba', n_jobs=-1)[:,1]

fpr, tpr, _ = roc_curve(y, probs_cv)
roc_auc = auc(fpr, tpr)

fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC (AUC={roc_auc:.3f})'))
fig_roc.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='chance', line=dict(dash='dash')) )
fig_roc.update_layout(title='Cross-validated ROC', xaxis_title='FPR', yaxis_title='TPR')
fig_roc.show()

# Probability distribution split by true label

# We create a small DataFrame for Plotly visuals
import pandas as pd

df_plot = X.copy()
df_plot['prob'] = probs_cv
df_plot['true'] = y.values

fig_prob = px.histogram(df_plot, x='prob', color='true', barmode='overlay', nbins=50, title='Predicted probability distribution (CV)')
fig_prob.update_traces(opacity=0.6)
fig_prob.show()

# Feature importance — logistic coefficients or RF importances if available
if hasattr(best_estimator, 'coef_'):
    coefs = best_estimator.coef_.ravel()
    feat_imp = pd.DataFrame({'feature': X.columns, 'importance': coefs})
    feat_imp['abs'] = feat_imp['importance'].abs()
    feat_imp = feat_imp.sort_values('abs', ascending=False).head(25)
    fig_coef = px.bar(feat_imp, x='importance', y='feature', orientation='h', title='Top 25 coefficients (absolute importance)')
    fig_coef.show()

elif hasattr(best_estimator, 'feature_importances_'):
    imps = best_estimator.feature_importances_
    feat_imp = pd.DataFrame({'feature': X.columns, 'importance': imps}).sort_values('importance', ascending=False).head(25)
    fig_imp = px.bar(feat_imp, x='importance', y='feature', orientation='h', title='Top 25 feature importances (RF)')
    fig_imp.show()

else:
    print('Estimator has no direct coefficient/importance attributes to display')

In [40]:
# 3d — Save / load helpers and batch predictions
#
# These helper routines load the final model + scaler (if present) and provide
# single-play and batch prediction helpers used by the dashboard and other scripts.
import joblib
import os

models_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'models')
model_path = os.path.join(models_dir, 'final_model.joblib')
scaler_path = os.path.join(models_dir, 'scaler.joblib')

print('Model files expected at:', model_path, scaler_path)

# Safety: load if available
if os.path.exists(model_path) and os.path.exists(scaler_path):
    final_model = joblib.load(model_path)
    final_scaler = joblib.load(scaler_path)
    print('Loaded final_model and scaler')
else:
    final_model, final_scaler = None, None
    print('Model files not found in models/ — run the training cell to generate them.')


def predict_single(play_map, model=final_model, scaler=final_scaler, features=X.columns.tolist(), threshold=0.45):
    """Return probability and 0/1 decision for a single play mapping using the saved model and scaler.

    The `play_map` should use the pre-play keys such as ydstogo, yardline_100, qtr, game_seconds_remaining, score_differential,
    shotgun, no_huddle, qb_dropback, and optionally pass_length and pass_location (values used to set one-hot columns).
    """
    if model is None or scaler is None:
        raise RuntimeError('Model or scaler not loaded. Run the training cell or create model files in models/.')

    ordered = np.zeros(len(features), dtype=float)
    for i,fname in enumerate(features):
        if fname in play_map:
            ordered[i] = float(play_map[fname])
        elif '_' in fname:
            base,cat = fname.split('_',1)
            if base in play_map and str(play_map[base]) == cat:
                ordered[i] = 1.0

    scaled = scaler.transform(ordered.reshape(1,-1))
    prob = model.predict_proba(scaled)[0,1]
    return {'probability': float(prob), 'decision': int(prob >= threshold)}


# Batch prediction helper for a DataFrame of plays
def predict_batch(df_plays, model=final_model, scaler=final_scaler, features=X.columns.tolist(), threshold=0.45):
    """Given a DataFrame of play columns or dict of plays, return probs and decisions for each row."""
    if model is None or scaler is None:
        raise RuntimeError('Model or scaler not loaded. Run the training cell or create model files in models/.')

    # Prepare ordered numeric matrix
    ordered_matrix = np.zeros((len(df_plays), len(features)), dtype=float)
    for i, fname in enumerate(features):
        if fname in df_plays.columns:
            ordered_matrix[:, i] = df_plays[fname].astype(float).fillna(0).values
        elif '_' in fname:
            base, cat = fname.split('_', 1)
            if base in df_plays.columns:
                ordered_matrix[:, i] = (df_plays[base].astype(str) == cat).astype(float).values
            else:
                ordered_matrix[:, i] = 0.0

    scaled = scaler.transform(ordered_matrix)
    probs = model.predict_proba(scaled)[:, 1]
    decisions = (probs >= threshold).astype(int)
    out = df_plays.copy()
    out['pred_prob'] = probs
    out['pred_decision'] = decisions
    return out

Model files expected at: c:\Users\owens\OneDrive\Desktop\BHDAC\models\final_model.joblib c:\Users\owens\OneDrive\Desktop\BHDAC\models\scaler.joblib
Loaded final_model and scaler


In [41]:
# Example usage of batch prediction on the validated dataset (show test sample predictions)
if final_model is not None and final_scaler is not None:
    sample_preview = X.sample(10, random_state=RANDOM_SEED).copy()
    preview_out = predict_batch(sample_preview)
    display(preview_out)
else:
    print('Model not available; run training cell to generate models.')

c:\Users\owens\miniconda3\envs\bhdac\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



,ydstogo,yardline_100,qtr,game_seconds_remaining,score_differential,shotgun,no_huddle,qb_dropback,pass_length_NA,pass_length_deep,pass_length_short,pass_location_NA,pass_location_left,pass_location_middle,pass_location_right,pred_prob,pred_decision
357869,5,30.0,1,3214.0,0.0,0,0,0,True,False,False,True,False,False,False,0.217525,0
419012,7,12.0,2,2110.0,4.0,0,0,1,False,False,True,False,False,True,False,0.352073,0
230084,4,36.0,4,306.0,-7.0,1,0,1,False,False,True,False,True,False,False,0.522266,1
131232,8,16.0,4,142.0,-20.0,1,0,1,False,True,False,False,True,False,False,0.195801,0
39565,10,38.0,4,154.0,-3.0,1,0,1,True,False,False,True,False,False,False,0.127439,0
81210,1,52.0,2,2397.0,-6.0,0,0,0,True,False,False,True,False,False,False,0.776251,1
298532,11,36.0,4,69.0,9.0,0,0,0,True,False,False,True,False,False,False,0.118462,0
190719,2,7.0,2,2182.0,-4.0,1,0,1,False,False,True,False,True,False,False,0.477637,1
15817,7,77.0,4,72.0,-27.0,1,0,1,True,False,False,True,False,False,False,0.145909,0
166694,2,77.0,4,209.0,-12.0,1,0,1,True,False,False,True,False,False,False,0.499561,1


### 3a — Single‑play prediction helper

I add a small helper that accepts a mapping or pandas Series for a single play, converts it into the exact feature order the model expects, applies the scaler, and returns both the predicted probability and a binary decision at the 0.45 threshold. This is the routine used by the dashboard and any production/interactive code to make consistent predictions.

In [42]:
# Save the final ordered feature columns so the prediction helper can build inputs in the same order
FEATURES = X.columns.tolist()

def predict_go_for_it(play_features, *, model=model, scaler=scaler, features=FEATURES, threshold=0.45):
    """
    Given a single play's features, return the model's probability and a binary decision (1 or 0).

    Parameters
    - play_features: dict-like or pandas Series with keys matching `features` (e.g. 'yards_to_go')
    - model: trained sklearn LogisticRegression instance
    - scaler: fitted StandardScaler used during training
    - features: ordered list of feature names
    - threshold: probability threshold to convert probability into a binary decision

    Returns
    - dict with keys: 'probability' (float), 'decision' (0 or 1)

    Behavior
    - We ensure the input is converted to the same order/shape the model expects.
    - We return `1` if probability >= threshold meaning "go for it"; otherwise `0`.
    """

    # Accept pandas Series, dict, or mapping.
    # We'll produce a single-row input that matches the model's feature ordering. The user can provide a dict or Series
    # containing pre-play fields (e.g., 'ydstogo', 'yardline_100', 'qtr', 'game_seconds_remaining', 'score_differential',
    # 'shotgun', 'no_huddle', 'qb_dropback', optionally 'pass_length' and 'pass_location').
    # The method will convert this input into the same columns the model was trained on (including one-hot columns),
    # filling unknowns with zero.

    # Convert mapping-like input into a dict for simple access
    if isinstance(play_features, pd.Series):
        play_map = play_features.to_dict()
    else:
        play_map = dict(play_features)

    # Build a 1D array matching the trained feature order. Start all zeros and then fill values where provided.
    ordered_values = np.zeros(len(features), dtype=float)

    for i, fname in enumerate(features):
        # For one-hot encoded categorical columns we expect names like 'pass_length_short' or 'pass_location_left'
        if fname in play_map:
            ordered_values[i] = float(play_map[fname])
        else:
            # Handle one-hot features: set to 1 when the user provides the base categorical value
            if '_' in fname:
                # expected pattern from get_dummies: <col>_<category>
                base, cat = fname.split('_', 1)
                if base in play_map and str(play_map[base]) == cat:
                    ordered_values[i] = 1.0
                # keep 0 otherwise
            else:
                # if play_map contains a numeric field named like the feature we already handled it
                # otherwise keep default 0
                pass

    values = ordered_values.reshape(1, -1)

    # Apply scaler (use exactly the same scaler fit on training set)
    values_scaled = scaler.transform(values)

    # Predict probability for class 1
    prob = model.predict_proba(values_scaled)[0, 1]

    # Convert to a binary decision at the specified threshold (default 0.45)
    decision = int(prob >= threshold)

    return {'probability': float(prob), 'decision': decision}


# Example usage: show a play and the model's decision
example_play = {
    'ydstogo': 4,
    'yardline_100': 55,           # this means 55 yards from own end => roughly 45 yardline in the old naming
    'qtr': 4,
    'game_seconds_remaining': 120,
    'score_differential': 3,
    'shotgun': 0,
    'no_huddle': 0,
    'qb_dropback': 1,
    # Optional categorical values for pass plays: 'short', 'deep', 'NA' (if run)
    'pass_length': 'short',
    'pass_location': 'left'
}

result = predict_go_for_it(example_play)
print("Example play prediction:", result)

Example play prediction: {'probability': 0.4002512139867655, 'decision': 0}


c:\Users\owens\miniconda3\envs\bhdac\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



## 4 — Sanity checks and quick visualizations

I run a few quick sanity checks on the held‑out test set — count how many plays the model classifies as "go" vs "no‑go" at 45%, and inspect notable edge cases visually. These quick checks help confirm the model's outputs make sense before using the model in a dashboard or other tools.

In [43]:
# Compute decisions across the test set using the helper for demonstration (vectorized route below)
# Use the new 45% threshold for decisions to match project policy
test_decisions = (y_pred_prob >= 0.45).astype(int)

# Decision counts
unique_decisions, counts = np.unique(test_decisions, return_counts=True)
decision_counts = dict(zip(unique_decisions.astype(str), counts.tolist()))
print("Decision counts on test set (0 = do not go / 1 = go) at 45% threshold:", decision_counts)


Decision counts on test set (0 = do not go / 1 = go) at 45% threshold: {np.str_('0'): 408, np.str_('1'): 773}


In [44]:
# Optional: compute and save statistically-significant or important-features automatically
# This helper will run the selection utility we added and write models/features.joblib
# Run this cell to override the currently saved features with a p-value / coefficient based selection.
try:
    from bhdacind1.select_features import main as _select_main
    _select_main()
except Exception as e:
    print('Unable to run selection helper directly from notebook; you can run from shell instead:', e)
    print('\nCommand to run from repo root:')
    print('\tpython -m bhdacind1.select_features')


Unable to run selection helper directly from notebook; you can run from shell instead: No module named 'bhdacind1'

Command to run from repo root:
	python -m bhdacind1.select_features
